# Python for Heliophysics

Today we'll briefly cover a few data structures I didn't get to last time, and then look at functionality from [Astropy](https://docs.astropy.org/en/stable/), [PlasmaPy](https://docs.plasmapy.org/en/stable/), and finally [SunPy](https://docs.sunpy.org/en/stable/).  The intention of this tutorial is not to be comprehensive, but rather to give a general idea of the capabilities of the different packages.  The [Python in Heliophysics Community (PyHC)](http://heliopython.org) is an effort to coordinate the development of multiple Python packages for heliophysics.   

## Preliminary imports

Please run the following cell with shift-enter which will import what we need for today.

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=Warning)

# General packages

import numpy as np
import matplotlib.pyplot as plt

# Settings for plotting

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 8)

# Astropy imports

import astropy.units as u
import astropy.constants as const
from astropy.time import Time
from astropy.coordinates import get_body_barycentric, SkyCoord

# PlasmaPy imports

from plasmapy.atomic import Particle

# SunPy imports

import sunpy.map
from sunpy.net import Fido, attrs
from sunpy import timeseries
from sunpy.coordinates import frames
from sunpy.coordinates import get_body_heliographic_stonyhurst

## Tuples, sets, and dictionaries

These are three different data types that are pretty common in Python.  We'll go over the essentials so that we can recognize them if we encounter them, and have an idea of what they can do.

### Tuples

A `tuple` is like a `list`, except that it is _immutable_.  A `list` is surrounded by square brackets, while a `tuple` is surrounded by parentheses.

In [ ]:
sample_tuple = (3, 'three')

In [ ]:
sample_tuple[0]

In [ ]:
sample_tuple[1]

Because a `tuple` is _immutable_, we cannot change which objects are inside of it.

In [ ]:
sample_tuple[1] = 'cat'

But we should be careful: we can still change mutable objects _within_ a `tuple`.

In [ ]:
sample_list = []
sample_tuple = (sample_list, 5)
print(sample_tuple)

In [ ]:
sample_list.append('new item in list')
print(sample_tuple)

### Sets

A `set` is a collection that behaves like a mathematical set.  If you put in more than one equal object in a set, only one of that object will show up in the set.  Sets are _mutable_ and not ordered.  Sets can be created with curly brackets.

In [ ]:
sample_set = {1, 3, 3, 3, 9, 9, 9, 9, 9}
print(sample_set)

We can perform standard operations on Python sets:

In [ ]:
multiples_of_two = {2, 4, 6, 8, 10, 12}
multiples_of_three = {3, 6, 9, 12}

The _difference_ provides objects that are in the first set but not in the second.

In [ ]:
multiples_of_two - multiples_of_three

The _intersection_ (denoted by `&`) provides objects that are in both sets.

In [ ]:
multiples_of_two & multiples_of_three

The union (denoted by `|`) provides all objects that are in either set.

In [ ]:
multiples_of_two | multiples_of_three

### Dictionaries

A `dict` matches _keys_ to _values_.  Here is a dictionary where
 
 - keys are atomic symbols
 - values are element names
 
The keys do not need to be strings, or even all of the same type.  In general, keys should be immutable.

In [ ]:
elements = {'H': 'hydrogen', 'He': 'helium', 'Li': 'lithium'}

In [ ]:
elements['H']

In [ ]:
elements['He']

We can loop over the keys and values in a dictionary.  An _item_ is a _key_ paired with the corresponding _value_.

In [ ]:
for key, value in elements.items():
    print(key, value)

We can add new items into dictionaries.

In [ ]:
elements[79] = 'gold'

## Astropy

Astropy is a Python package that contains essential functionality needed by most astronomers.  We'll look at three of the subpackages that get used a lot in other packages.

### `astropy.units`

In [ ]:
import astropy.units as u
import astropy.constants as const

We can hit the tab button or use `dir` to show what's in `astropy.units` and `astropy.constants`.

In [ ]:
distance = 42 * u.m
print(distance)

In [ ]:
distance.to('cm')

In [ ]:
distance.to(u.cm)

In [ ]:
distance.to(u.imperial.mile)

When we multiply a number with a unit, we create a `Quantity` object.  We can also create a `Quantity` directly.

In [ ]:
time = u.Quantity(42, 's')
print(time)

`astropy.units` will raise an error if we try to convert a `Quantity` into something with incompatible units.  Remember to scroll to the bottom of the traceback.

In [ ]:
time_interval.to('kg')

We can do operations with units!

In [ ]:
velocity = distance / time
print(velocity)

We can access the value and unit directly

In [ ]:
velocity.value

In [ ]:
velocity.unit

`astropy.units` will raise an error if we try to do operations with incompatible units.  This is kind of like an additional test.  Again, scroll to the bottom of the traceback.

In [ ]:
distance + velocity

We don't need to have the units be identical.  Rather, they just need to be compatible

In [ ]:
distance_in_cm = 1e18 * u.cm
distance_in_pc = 4 * u.pc
print(distance_in_cm + distance_in_pc)

In [ ]:
distance_in_pc.si

In [ ]:
distance_in_pc.cgs

In [ ]:
distance_in_cm.to_string()

We can also create custom units

In [ ]:
mph = u.imperial.mile / u.hour
88 * u.mph

We can even use `astropy.units` to help write cookbooks with ridiculous units.

In [ ]:
(u.barn * u.Mpc).to(u.imperial.tsp)

A barn-megaparsec is roughly a teaspoon!

**Takeaway point:** `astropy.units` is extremely helpful when we're working with physical quantities that have units!

### `astropy.constants`

The `astropy.constants` subpackage contains the most commonly needed physical constants for physics and astronomy.

In [ ]:
import astropy.constants as const

In [ ]:
const.c

In [ ]:
const.G

In [ ]:
const.L_sun

In [ ]:
const.R_sun

In [ ]:
const.M_sun

NumPy also contains some mathematical constants and special values.

In [ ]:
np.pi

In [ ]:
np.inf > 999999999999999999999999

NumPy has a special value referring to "Not A Number".

In [ ]:
np.nan

`np.nan` shows up when we try to do math that makes the universe grumpy.

In [ ]:
np.inf - np.inf

### `astropy.time`

In [ ]:
from astropy.time import Time

In [ ]:
times = ['1999-01-01T00:00:00.123456789', '2010-01-01T00:00:00']

t = Time(times, format='isot', scale='utc')

We can print out the time in the 'iso' standard format

In [ ]:
t.iso

We can convert to many different time & date formats, such as the Julian Date.

In [ ]:
t.jd

In [ ]:
dt = t[1] - t[0]

`Time` objects also let us do conversions that I haven't thought about since grad school since I'm a theorist.

Greenwich Apparent Sidereal Time (GAST) is Greenwich Mean Sidereal Time (GMST) corrected for the shift in the position of the vernal equinox due to nutation.

In [ ]:
t.sidereal_time('apparent', 'greenwich')

## PlasmaPy

PlasmaPy is a package that intends to be for plasma physics what Astropy is for astronomy.  PlasmaPy is much newer than Astropy, but the PlasmaPy community has been gaining community support.  This is the package I spend most of my time on.

### `plasmapy.atomic`

We'll cover one of the most mature subpackages in PlasmaPy 

In [ ]:
from plasmapy.atomic import Particle

In [ ]:
electron = Particle("e-")
proton = Particle("p+")
singly_ionized_iron = Particle("Fe 1+")

In [ ]:
Particle("p+") == Particle("H-1 1+") == Particle("proton")

In [ ]:
electron.mass

In [ ]:
electron.charge

In [ ]:
electron.mass_energy.to('MeV')

In [ ]:
singly_ionized_iron.atomic_number

In [ ]:
singly_ionized_iron.recombine()

# SunPy

In [ ]:
import sunpy.map
from sunpy.net import Fido, attrs
import astropy.units as u
from sunpy import timeseries
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 8)

## `Fido` lets us search for and download many solar data sets

`Fido` provides an alternative way to access the _Virtual Solar Observatory_.

Let's start with a search for SDO/AIA data.

In [ ]:
time_interval = attrs.Time('2014/05/15 08:00', '2014/05/15 08:10')
instrument = attrs.Instrument('AIA')

results = Fido.search(time_interval, instrument)

print(results.file_num)

In [ ]:
results

In [ ]:
wavelength = attrs.Wavelength(171 * u.Angstrom)
cadence = attrs.vso.Sample(2 * u.minute)

results = Fido.search(time_interval, instrument, wavelength, cadence)

In [ ]:
results

Let's download the files by calling `Fido.fetch`.  For the moment, let's put them in our current working directory.

In [ ]:
aia_files = Fido.fetch(results, path='.')

In [ ]:
aia_files

### Times series data: a GOES/XRS X-ray curve

In [ ]:
goes_time_interval = attrs.Time('2013-10-28 00:00', '2013-10-28 12:00')
goes = attrs.Instrument('XRS')

search_results = Fido.search(goes_time_interval, goes)
goes_files = Fido.fetch(search_results[0])

In [ ]:
goes_lc = timeseries.TimeSeries(goes_files)

Let's take a look.  There be an X-flare here!

In [ ]:
goes_lc.peek()

In [ ]:
goes_lc.meta

In [ ]:
goes_lc.units

### Maps in SunPy

In [ ]:
stereo = (
    attrs.vso.Source('STEREO_A') &
    attrs.Instrument('EUVI') &
    attrs.Time('2010-08-19', '2010-08-19T00:10:00')
)

aia = (
    attrs.Instrument('AIA') &
    attrs.vso.Sample(24 * u.hour) &
    attrs.Time('2010-08-19', '2010-08-19T00:10:00')
)

wave = attrs.Wavelength(17 * u.nm, 18 * u.nm)

res = Fido.search(wave, aia | stereo)

In [ ]:
files = Fido.fetch(res)

We use `sunpy.map.Map` to generate coordinate aware 2D images.  Let's create a `Map` each for the AIA and STEREO observations.

In [ ]:
map_aia, map_stereo = sunpy.map.Map(sorted(files))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1, projection = map_aia)
map_aia.plot(axes = ax1)
ax2 = fig.add_subplot(1,2,2, projection = map_stereo)
map_stereo.plot(axes = ax2)

In [ ]:
ax = plt.subplot(projection='polar')

plt.polar(
    map_stereo.observer_coordinate.lon.to(u.rad), 
    map_stereo.observer_coordinate.radius.to(u.AU), 
    marker='o',  
    ms=10, 
    label='STEREO_A',
)

plt.polar(
    map_aia.observer_coordinate.lon.to(u.rad), 
    map_aia.observer_coordinate.radius.to(u.AU), 
    marker='o', 
    ms=10, 
    label='AIA',
)

plt.polar(
    sun_pos.lon.to(u.rad), 
    sun_pos.radius.to(u.AU), 
    'o', ms=20, 
    label='Sun', 
    color='yellow',
)


ax.set_theta_zero_location("S")

plt.title('Position of the Sun, AIA and Stereo')
plt.legend()